# Universidad Nacional de Córdoba
# DIPLODATOS - Facultad de Matemática, Astronomía, Física y Computación

## Trabajo Práctico Número 1 (Analisis del archivo exercise_1.py)

Autores: 
* Diaz Cobos Facundo
* Epifanio Luis
* Gonzalez Leonardo David
* Gualpa Mariano Martín




# 1. Consigna del Ejercicio 1.

1. Procesar el conjunto de datos para obtener una representación TfIDf de cada review.
  * Hint: User el código de *Aprendizaje Supervisado*
  * Hint: El método de fit de Keras crea internamente un conjunto de datos de validación, por lo que no tienen que preocuparse por eso.

2. Construir un pipeline de clasificación con un modelo Keras MLP.

3. Entrenar uno o varios modelos (con dos o tres es suficiente, veremos más de esto en el práctico 2). Evaluar los modelos en el conjunto de test.

4. Reportar los hyperparámetros y resultados de todos los modelos entrenados. Para esto, pueden utilizar una notebook o un archivo (pdf|md). Dentro de este reporte tiene que describir:
  * Hyperparámetros con los que procesaron el dataset: tamaño del vocabulario, normalizaciones, etc.
  * Las decisiones tomadas al construir cada modelo: regularización, dropout, número y tamaño de las capas, optimizador.
  * Proceso de entrenamiento: división del train/test, tamaño del batch, número de épocas, métricas de evaluación. Seleccione los mejores hiperparámetros en función de su rendimiento. El proceso de entrenamiento debería ser el mismo para todos los modelos.
  * (Punto estrella) Analizar si el clasificador está haciendo overfitting. Esto se puede determinar a partir del resultado del método fit.

# 2. Desarrollo.

## 2.1. Carga de Librerías y Funciones 

In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [2]:
# Exercise 1
import argparse
import pandas
import numpy as np
import datetime
import os

from keras.models import Sequential
from tensorflow.python.keras.utils import np_utils   # for tf 1.3.1
#from tensorflow.python.keras import utils as np_utils     # for tf 1.4.1

from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from itertools import repeat
from IPython.display import display, HTML

from printutils import print_message, print_new_process, print_end


/Users/mgualpa/anaconda3/envs/DataScience/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def read_args():
    parser = argparse.ArgumentParser(description='Exercise 1')
    # Here you have some examples of classifier parameters. You can add
    # more arguments or change these if you need to.
    parser.add_argument('--num_units', nargs='+', default=[100], type=int,
                        help='Number of hidden units of each hidden layer.')
    parser.add_argument('--dropout', nargs='+', default=[0.5], type=float,
                        help='Dropout ratio for every layer.')
    parser.add_argument('--batch_size', type=int, default=32,
                        help='Number of instances in each batch.')

    # New parameters:    
    parser.add_argument('--model', type=int, default=10, help='Number of model to run')
    parser.add_argument('--max_features', type=int, default=2000, help='Max number of words used inTfidfVectorizer')
    parser.add_argument('--epochs', type=int, default=10, help='Number of epochs')
    parser.add_argument('--shuffle', type=str, default='batch', help='Shuffle value')
    parser.add_argument('--random_seed', type=int, default=10, help='Random seed number')
    parser.add_argument('--verbose', type=int, default=1, help='Verbose info on screen')
    
    # parse parameters
    if arguments == None:
        args = parser.parse_args()
    else:
        args = parser.parse_args(arguments)

    assert len(args.num_units) == len(args.dropout)
    return args

## 2.2. Procesar el conjunto de datos utilizando TfIDf vectorizer para crear una matriz de entrada.

__Responde al punto:__

_1. Procesar el conjunto de datos para obtener una representación TfIDf de cada review._
  * _Hint: User el código de *Aprendizaje Supervisado*_
  * _Hint: El método de fit de Keras crea internamente un conjunto de datos de validación, por lo que no tienen que preocuparse por eso._


### Apply the Tfidf vectorizer to create input matrix
Creamos un método para vectorizar la entrada:

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
    
# TODO 1: Apply the Tfidf vectorizer to create input matrix
def vectorize_input(x_train, x_test, args):

    # vectorizer = TfidfVectorizer(analyzer='word', use_idf=True, max_features=args.max_features) # Original
    vectorizer = TfidfVectorizer(binary=False, max_df=0.1, min_df=5, ngram_range=(1, 4), max_features=args.max_features)
    
    x_train_vec = vectorizer.fit_transform(x_train).toarray()
    print_message('x_train_vec - type: {}, shape:{}'.format(type(x_train_vec),x_train_vec.shape),args)
    
    #x_test_vec = vectorizer.fit_transform(x_test).toarray()
    x_test_vec = vectorizer.transform(x_test).toarray()

    print_message('x_test_vec - type: {}, shape:{}'.format(type(x_test_vec),x_test_vec.shape),args)
      
    return x_train_vec, x_test_vec


#### Funcion: load_dataset():         
La misma función del archivo, donde llamamos a nuestra nueva rutina:

In [5]:
from keras.datasets import imdb

def load_dataset(args):
    
    print_new_process('Load and vectorize Data:',args)
    
    dataset = load_files('dataset/txt_sentoken', shuffle=False)

    #(X_train, X_test), (y_train, y_test) = imdb.load_data()
    
    X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.25, random_state=10042)
    
    print_message('Training samples {} ({}), test_samples {} ({})'.format(
        len(X_train),
        len(y_train), 
        len(X_test), 
        len(y_test)), args)

    # TODO 1: Apply the Tfidf vectorizer to create input matrix
    x_train_vec, x_test_vec = vectorize_input(X_train, X_test,args)
        
    return x_train_vec, x_test_vec, y_train, y_test

## 2.3. Construir modelos Keras

Se plantean modelos con diferentes arquitecturas para realizar las pruebas:

### Build the Keras models:

### Model 1: 

In [12]:
from keras.models import Sequential 
from keras.layers import Dense, Activation, Dropout, Embedding, LSTM
from keras import optimizers, regularizers

def build_keras_model_1(x_train_vec, args):
    
    input_size = x_train_vec.shape[1]
    num_units = args.num_units[0]   
    
    model = Sequential()
    
    #model.add(Dense(1, activation = 'sigmoid', input_shape=(input_size,)))

    model.add(Dense(num_units, activation='relu', input_shape=(input_size,)))
    
    model.add(Dropout(args.dropout[0]))    
    
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=['binary_accuracy'])
    
    # Show model info on screen
    print_message('MODEL 1:', args )
    print_message( model.summary(), args )

    """
    input_size = x_train_vec.shape[1]
       
    model = Sequential()
    model.add(Dense( args.num_units[0], input_shape=(input_size,)))    
    model.add(Activation('relu'))
    
    model.add(Dropout(args.dropout[0]))    
    
    model.add(Dense(2))
    model.add(Activation('sigmoid'))
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=optimizers.Adagrad(), 
                  metrics=['accuracy'])
    
    # Show model info on screen
    print_message('MODEL 1:', args )
    print_message( model.summary(), args )


    """
    
    
    
    return model

### Model 2: 

In [13]:
def build_keras_model_2(x_train_vec, args):
    
    input_size = x_train_vec.shape[1]
       
    model = Sequential()

    model.add(Dense(input_size, kernel_regularizer=regularizers.l1(0.001), activation='relu', input_shape=(input_size,)))
    model.add(Dropout(0.5))
    model.add(Dense(16, kernel_regularizer=regularizers.l1(0.001),activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Use of REGULARIZATION
    #model = models.Sequential()
    #model.add(Dense(16, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),activation='relu', input_shape=(10000,)))
    #model.add(Dense(16, kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001),activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))

    # REGULARIZERS L1 L2
    #regularizers.l1(0.001)
    #regularizers.l2(0.001)
    #regularizers.l1_l2(l1=0.001, l2=0.001)
    
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Show model info on screen
    print_message('MODEL 2:', args )
    print_message( model.summary(), args )

    return model

"""

def build_keras_model_2(x_train_vec, args):
    
    input_size = x_train_vec.shape[1]
       
    model = Sequential()

    model.add(Dense(input_size, input_dim=input_size, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(args.dropout[0]))
    model.add(Dense(int(input_size/2), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(args.dropout[0]))
    model.add(Dense(int(input_size), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(args.dropout[0]))
    model.add(Dense(int(input_size/2), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(args.dropout[0]))
    model.add(Dense(2, kernel_initializer='normal', activation='sigmoid'))
    
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
    
    # Show model info on screen
    print_message('MODEL 2:', args )
    print_message( model.summary(), args )

    return model
    
"""

"\n\ndef build_keras_model_2(x_train_vec, args):\n    \n    input_size = x_train_vec.shape[1]\n       \n    model = Sequential()\n\n    model.add(Dense(input_size, input_dim=input_size, kernel_initializer='normal', activation='relu'))\n    model.add(Dropout(args.dropout[0]))\n    model.add(Dense(int(input_size/2), kernel_initializer='normal', activation='relu'))\n    model.add(Dropout(args.dropout[0]))\n    model.add(Dense(int(input_size), kernel_initializer='normal', activation='relu'))\n    model.add(Dropout(args.dropout[0]))\n    model.add(Dense(int(input_size/2), kernel_initializer='normal', activation='relu'))\n    model.add(Dropout(args.dropout[0]))\n    model.add(Dense(2, kernel_initializer='normal', activation='sigmoid'))\n    \n    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])\n    \n    # Show model info on screen\n    print_message('MODEL 2:', args )\n    print_message( model.summary(), args )\n\n    return model\n    \n"

### Model 3:

In [14]:
def build_keras_model_3(x_train_vec, args):
    
    input_size = x_train_vec.shape[1]
    
    model = Sequential()
    model.add(Embedding(input_size, args.num_units[0], input_shape=(input_size,)))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    
    # Show model info on screen
    print_message('MODEL 3:', args )
    print_message( model.summary(), args )

    return model

### Model 4

In [15]:
from keras.layers import Dense, Activation, Dropout, Embedding, LSTM, Flatten

def build_keras_model_4(x_train_vec, args):
 
    input_size = x_train_vec.shape[1]
       
    model = Sequential()
    model.add(Dense(args.num_units[0], input_shape=(input_size,)))    
    model.add(Activation('relu'))
    
    model.add(Dropout(args.dropout[0]))    
    
    model.add(Dense( int(input_size/2) ))    
    model.add(Activation('relu'))
    
    model.add(Dropout(args.dropout[0]))       

    model.add(Dense( int(input_size/4) ))    
    model.add(Activation('relu'))
    
    model.add(Dropout(args.dropout[0]))
    
    model.add(Dense(2))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer=optimizers.Adagrad(), 
                  metrics=['accuracy'])
    
    # Show model info on screen
    print_message('MODEL 4:', args )
    print_message( model.summary(), args )
    
    return model


## 2.4. Entrenamiento del Modelo

### 2.4.1. Main:

Función principal que llama a las rutinas anteriores para realizar el entrenamiento. 

__Responde al punto:__

_2. Construir un pipeline de clasificación con un modelo Keras MLP._

In [16]:
from keras.utils import to_categorical

def main():

    USE_CATEGORICAL = False
    
    experiment_number = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    
    args = read_args()
    
    print_new_process('START:', args)
    
    # Configuramos la semilla aleatoria por reproducibilidad
    np.random.seed(args.random_seed)
    
    # Cargamos el dataset
    x_train_vec, x_test_vec, y_train, y_test_orginal = load_dataset(args)

    # TODO 2: Convert the labels to categorical
    if (USE_CATEGORICAL):
        num_classes = 2 
        #y_train_cat = np_utils.to_categorical(y_train, num_classes)    
        #y_test_cat = np_utils.to_categorical(y_test_orginal, num_classes)    
        y_train_cat = to_categorical(y_train)    
        y_test_cat = to_categorical(y_test_orginal)    

    else:
        y_train_cat = y_train    
        y_test_cat = y_test_orginal    

    
    
    print_new_process('Build Model:', args )
    # TODO 3: Build the Keras model
    switcher = {
        1: build_keras_model_1,
        2: build_keras_model_2,
        3: build_keras_model_3,
        4: build_keras_model_4,
    }
    # Get the function from switcher dictionary
    model_builder = switcher.get(args.model, lambda: "nothing")
    
    model = model_builder(x_train_vec, args)
    
    # TODO 4: Fit the model    
    print_new_process('Fit:', args)
    history = model.fit(x_train_vec, y_train_cat,
                        batch_size=args.batch_size,
                        epochs=args.epochs,
                        shuffle=args.shuffle,
                        verbose=1,
                        validation_split=0.70
             )

    # TODO 5: Evaluate the model, calculating the metrics.
    # Option 1: Use the model.evaluate() method. For this, the model must be
    # already compiled with the metrics.
    print_new_process('Predictions:',args)
    predictions = history.model.predict_classes(x_test_vec, verbose=1)
    
    if args.verbose == 1:
        display(str(list( predictions)))

    print_new_process('Test:',args)
    if args.verbose == 1:        
        display(str(list(y_test_orginal)))
    
    print_new_process('Performance:',args)
    score, accuracy = history.model.evaluate(x_test_vec, y_test_cat)
    #score, accuracy = model.evaluate(x_train_vec, y_train_cat)

    print_message('[score, accuracy]', args)
    print_message([score, accuracy], args)

    # Option 2: Use the model.predict() method and calculate the metrics using
    # sklearn. We recommend this, because you can store the predictions if
    # you need more analysis later. Also, if you calculate the metrics on a
    # notebook, then you can compare multiple classifiers.
    # predictions = ...
    # performance = ...

    # TODO 6: Save the results.    
    parameters_path = 'results/parameters/'
    if not os.path.exists(parameters_path):
        os.makedirs(parameters_path)
    predictions_path = 'results/predictions/'
    if not os.path.exists(predictions_path):
        os.makedirs(predictions_path)

    parameters_filename = 'results/parameters/parameters_{0}_acc_{1:.6f}.csv'.format(experiment_number, accuracy)
    print_new_process('Saving parameters: {}'.format(parameters_filename),args)  
    
    parameters_df = pandas.DataFrame(columns=['Parameter','Value'])
    for k,v in sorted(vars(args).items()):
        row = pandas.Series([str(k), str(v)], index=['Parameter', 'Value'])
        parameters_df = parameters_df.append(row,ignore_index=True)
    parameters_df.to_csv(parameters_filename, index=False )
    
    predictions_filename = 'results/predictions/predictions_{0}_acc_{1:.6f}.csv'.format(experiment_number, accuracy)
    print_new_process('Saving predictions: {}'.format(predictions_filename),args)  
    predictions_df = pandas.DataFrame(y_test_orginal, columns=['true_label'])
    predictions_df.loc[:, 'predicted'] = predictions
    predictions_df.to_csv(predictions_filename, index=False)
    
    print_message('Done.',args)    
    print_end('STOP.',args)    


<br/>

### 2.4.2. Ejecución del modelo:

__Responde al punto:__

_3. Entrenar uno o varios modelos (con dos o tres es suficiente, veremos más de esto en el práctico 2). Evaluar los modelos en el conjunto de test._


Ejecutamos una prueba de main para poder visualizar que el funcionamiento sea adecuado:

In [19]:
# Cargamos argumentos de prueba
arguments = ['--model=1',
             '--max_features=6000',
             '--num_units=100',
             '--dropout=0.5',
             '--batch_size=200',             
             '--epochs=200',
             '--shuffle=batch',
             '--random_seed=10',
             '--verbose=1'
            ]

"""
arguments = ['--model=1',
             '--max_features=4000',
             '--num_units=1',
             '--dropout=0.5',
             '--batch_size=200',             
             '--epochs=70',
             '--shuffle=batch',
             '--random_seed=10',
             '--verbose=1'
            ]
"""

# Ejecutamos el procedimiento principal
main()



2018-12-10 22:00:44 ------------------------------------------------------------------------
2018-12-10 22:00:44 - START:
2018-12-10 22:00:44 ------------------------------------------------------------------------
2018-12-10 22:00:44 - Load and vectorize Data:
2018-12-10 22:00:57 - Training samples 1500 (1500), test_samples 500 (500)
2018-12-10 22:01:12 - x_train_vec - type: <class 'numpy.ndarray'>, shape:(1500, 6000)
2018-12-10 22:01:13 - x_test_vec - type: <class 'numpy.ndarray'>, shape:(500, 6000)
2018-12-10 22:01:13 ------------------------------------------------------------------------
2018-12-10 22:01:13 - Build Model:
2018-12-10 22:01:13 - MODEL 1:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 100)               600100    
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
____

450/450 [==============================] - 0s 280us/step - loss: 0.0658 - binary_accuracy: 1.0000 - val_loss: 0.4729 - val_binary_accuracy: 0.7743
Epoch 43/200
450/450 [==============================] - 0s 279us/step - loss: 0.0599 - binary_accuracy: 1.0000 - val_loss: 0.4719 - val_binary_accuracy: 0.7714
Epoch 44/200
450/450 [==============================] - 0s 278us/step - loss: 0.0592 - binary_accuracy: 1.0000 - val_loss: 0.4710 - val_binary_accuracy: 0.7705
Epoch 45/200
450/450 [==============================] - 0s 278us/step - loss: 0.0537 - binary_accuracy: 1.0000 - val_loss: 0.4701 - val_binary_accuracy: 0.7686
Epoch 46/200
450/450 [==============================] - 0s 276us/step - loss: 0.0518 - binary_accuracy: 1.0000 - val_loss: 0.4694 - val_binary_accuracy: 0.7686
Epoch 47/200
450/450 [==============================] - 0s 274us/step - loss: 0.0507 - binary_accuracy: 1.0000 - val_loss: 0.4687 - val_binary_accuracy: 0.7695
Epoch 48/200
450/450 [==============================]

450/450 [==============================] - 0s 276us/step - loss: 0.0123 - binary_accuracy: 1.0000 - val_loss: 0.4712 - val_binary_accuracy: 0.7695
Epoch 94/200
450/450 [==============================] - 0s 277us/step - loss: 0.0117 - binary_accuracy: 1.0000 - val_loss: 0.4715 - val_binary_accuracy: 0.7695
Epoch 95/200
450/450 [==============================] - 0s 275us/step - loss: 0.0116 - binary_accuracy: 1.0000 - val_loss: 0.4718 - val_binary_accuracy: 0.7695
Epoch 96/200
450/450 [==============================] - 0s 276us/step - loss: 0.0109 - binary_accuracy: 1.0000 - val_loss: 0.4721 - val_binary_accuracy: 0.7695
Epoch 97/200
450/450 [==============================] - 0s 274us/step - loss: 0.0104 - binary_accuracy: 1.0000 - val_loss: 0.4725 - val_binary_accuracy: 0.7686
Epoch 98/200
450/450 [==============================] - 0s 276us/step - loss: 0.0109 - binary_accuracy: 1.0000 - val_loss: 0.4728 - val_binary_accuracy: 0.7686
Epoch 99/200
450/450 [==============================]

450/450 [==============================] - 0s 278us/step - loss: 0.0055 - binary_accuracy: 1.0000 - val_loss: 0.4876 - val_binary_accuracy: 0.7695
Epoch 145/200
450/450 [==============================] - 0s 279us/step - loss: 0.0052 - binary_accuracy: 1.0000 - val_loss: 0.4879 - val_binary_accuracy: 0.7695
Epoch 146/200
450/450 [==============================] - 0s 322us/step - loss: 0.0051 - binary_accuracy: 1.0000 - val_loss: 0.4883 - val_binary_accuracy: 0.7705
Epoch 147/200
450/450 [==============================] - 0s 458us/step - loss: 0.0048 - binary_accuracy: 1.0000 - val_loss: 0.4886 - val_binary_accuracy: 0.7705
Epoch 148/200
450/450 [==============================] - 0s 396us/step - loss: 0.0047 - binary_accuracy: 1.0000 - val_loss: 0.4890 - val_binary_accuracy: 0.7705
Epoch 149/200
450/450 [==============================] - 0s 398us/step - loss: 0.0046 - binary_accuracy: 1.0000 - val_loss: 0.4893 - val_binary_accuracy: 0.7695
Epoch 150/200
450/450 [=========================

Epoch 195/200
450/450 [==============================] - 0s 285us/step - loss: 0.0031 - binary_accuracy: 1.0000 - val_loss: 0.5027 - val_binary_accuracy: 0.7695
Epoch 196/200
450/450 [==============================] - 0s 378us/step - loss: 0.0026 - binary_accuracy: 1.0000 - val_loss: 0.5030 - val_binary_accuracy: 0.7695
Epoch 197/200
450/450 [==============================] - 0s 454us/step - loss: 0.0026 - binary_accuracy: 1.0000 - val_loss: 0.5033 - val_binary_accuracy: 0.7695
Epoch 198/200
450/450 [==============================] - 0s 290us/step - loss: 0.0028 - binary_accuracy: 1.0000 - val_loss: 0.5036 - val_binary_accuracy: 0.7695
Epoch 199/200
450/450 [==============================] - 0s 374us/step - loss: 0.0026 - binary_accuracy: 1.0000 - val_loss: 0.5039 - val_binary_accuracy: 0.7714
Epoch 200/200
450/450 [==============================] - 0s 419us/step - loss: 0.0028 - binary_accuracy: 1.0000 - val_loss: 0.5042 - val_binary_accuracy: 0.7714
2018-12-10 22:01:42 --------------

'[array([1], dtype=int32), array([0], dtype=int32), array([0], dtype=int32), array([1], dtype=int32), array([1], dtype=int32), array([1], dtype=int32), array([0], dtype=int32), array([1], dtype=int32), array([0], dtype=int32), array([0], dtype=int32), array([0], dtype=int32), array([0], dtype=int32), array([1], dtype=int32), array([1], dtype=int32), array([1], dtype=int32), array([1], dtype=int32), array([1], dtype=int32), array([0], dtype=int32), array([0], dtype=int32), array([1], dtype=int32), array([1], dtype=int32), array([0], dtype=int32), array([0], dtype=int32), array([1], dtype=int32), array([0], dtype=int32), array([0], dtype=int32), array([0], dtype=int32), array([0], dtype=int32), array([0], dtype=int32), array([1], dtype=int32), array([0], dtype=int32), array([1], dtype=int32), array([0], dtype=int32), array([1], dtype=int32), array([0], dtype=int32), array([0], dtype=int32), array([1], dtype=int32), array([0], dtype=int32), array([0], dtype=int32), array([1], dtype=int32)

2018-12-10 22:01:42 ------------------------------------------------------------------------
2018-12-10 22:01:42 - Test:


'[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1,

2018-12-10 22:01:42 ------------------------------------------------------------------------
2018-12-10 22:01:42 - Performance:
500/500 [==============================] - 0s 169us/step
2018-12-10 22:01:42 - [score, accuracy]
2018-12-10 22:01:42 - [0.4475015070438385, 0.7999999990463257]
2018-12-10 22:01:42 ------------------------------------------------------------------------
2018-12-10 22:01:42 - Saving parameters: results/parameters/parameters_20181210220044_acc_0.800000.csv
2018-12-10 22:01:43 ------------------------------------------------------------------------
2018-12-10 22:01:43 - Saving predictions: results/predictions/predictions_20181210220044_acc_0.800000.csv
2018-12-10 22:01:43 - Done.
2018-12-10 22:01:43 ------------------------------------------------------------------------
2018-12-10 22:01:43 - STOP.
2018-12-10 22:01:43 ------------------------------------------------------------------------


In [ ]:






















# Cargamos argumentos de prueba
arguments = ['--model=2',
             '--max_features=6000',
             '--num_units=3000',
             '--dropout=0.2',
             '--batch_size=200',             
             '--epochs=30',
             '--shuffle=batch',
             '--random_seed=10',
             '--verbose=1'
            ]

# Ejecutamos el procedimiento principal
main()


In [ ]:
# Cargamos argumentos de prueba
arguments = ['--model=2',
             '--max_features=6000',
             '--num_units=300',
             '--dropout=0.2',
             '--batch_size=200',             
             '--epochs=30',
             '--shuffle=batch',
             '--random_seed=10',
             '--verbose=1'
            ]

# Ejecutamos el procedimiento principal
main()



# 3. Reporte 

__Responde al punto:__
_Reportar los hyperparámetros y resultados de todos los modelos entrenados. Para esto, pueden utilizar una notebook o un archivo (pdf|md). Dentro de este reporte tiene que describir:_

* _Hyperparámetros con los que procesaron el dataset: tamaño del vocabulario, normalizaciones, etc._
* _Las decisiones tomadas al construir cada modelo: regularización, dropout, número y tamaño de las capas, optimizador._
* _Proceso de entrenamiento: división del train/test, tamaño del batch, número de épocas, métricas de evaluación._ 
* _Seleccione los mejores hiperparámetros en función de su rendimiento. El proceso de entrenamiento debería ser el mismo
para todos los modelos._
* _(Punto estrella) Analizar si el clasificador está haciendo overfitting. Esto se puede determinar a partir del resultado del método fit._


## 3.1. Reporte de Hiperparámetros

Training samples 1500 (1500), test_samples 500 (500)
2018-12-08 17:36:49 - x_train_vec - type: <class 'numpy.ndarray'>, shape:(1500, 35393)
2018-12-08 17:36:50 - x_test_vec - type: <class 'numpy.ndarray'>, shape:(500, 22062)
                

## 3.2. Descripción de los Modelos Utilizados

Se describen a continuación los modelos utilizados para el trabajo.

### 3.2.1. Descripción del Modelo 1





### 3.2.2. Descripción del Modelo 2



### 3.2.3. Descripción del Modelo 3



### 3.2.4. Descripción del Modelo 4



## 3.3. Descripción de Proceso de Entrenamiento

## 3.4. Selección de los mejores hiperparámetros

## 3.5. Análisis sobre el Overfitting


# 4. Conclusiones

In [ ]:
dataset = load_files('dataset/txt_sentoken', shuffle=False)

X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.25, random_state=42)



In [ ]:
vectorizer = TfidfVectorizer(analyzer='word', use_idf=True, max_features=6000)
    
x_train_vec = vectorizer.fit_transform(X_train).toarray()
    
x_test_vec = vectorizer.fit_transform(X_test).toarray()
    

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split

from sklearn import metrics


def print_eval(model, X, y_true):
    y_pred = model.predict(X)
    acc = metrics.accuracy_score(y_true, y_pred)
    print('accuracy\t{:2.2f}\n'.format(acc))
    print(metrics.classification_report(y_true, y_pred, target_names=['neg', 'pos']))
    cm = metrics.confusion_matrix(y_true, y_pred)
    print(cm)


def eval(model, X, y_true):
    y_pred = model.predict(X)
    acc = metrics.accuracy_score(y_true, y_pred)
    f1 = metrics.f1_score(y_true, y_pred, average='macro')
    return {'acc': acc, 'f1': f1}


def print_short_eval(model, X, y_true):
    res = eval(model, X, y_true)
    print('accuracy\t{acc:2.6f}\tmacro f1\t{f1:2.6f}'.format(**res))
    return res
    


    
pipeline = Pipeline([
    #('vect', TfidfVectorizer(binary=False, max_df=0.1, min_df=5, ngram_range=(1, 4))),
    ('vect', TfidfVectorizer(analyzer='word', use_idf=True, max_features=6000)),
    ('clf', LogisticRegression(C=1, random_state=0, tol=0.001))
])



pipeline.fit(X_train, y_train)

print_short_eval(pipeline, X_train, y_train)
print_short_eval(pipeline, X_test, y_test)



In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(binary=True)

#vect = CountVectorizer(binary=True)
pipeline = Pipeline([
    ('vect', vect),
    ('clf', LinearSVC(random_state=0)),
])
pipeline.fit(X_train, y_train)
print_eval(pipeline, X_dev, y_dev)